## Run Qdrant in Docker

Chick if Qdrant Runing ...
http://localhost:6333/dashboard

In [6]:
from sentence_transformers import SentenceTransformer
import json
import numpy as np
from tqdm import tqdm

In [7]:


# Import client library
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

client = QdrantClient("http://localhost:6333")

In [8]:
model = SentenceTransformer(
    "all-MiniLM-L6-v2", device="cpu"
)  # or device="cpu" if you don't have a GPU

/Users/abdulwahabmac/Desktop/MyFiles/Projects/Training/Tuwaiq/LLM_lite_app/.env/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [80]:
collection_name = 'startups'

if not client.collection_exists(collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(
                                    size=model.get_sentence_embedding_dimension(),
                                    distance=Distance.COSINE
                                   ),
    )
    print(f"collection {collection_name} Created !!")
else:
    print(f"collection {collection_name} exists !!")

collection startups Created !!


In [81]:
fd = open("./startups_demo.json")
payload = list(map(json.loads, fd))

# Load all vectors into memory, numpy array works as iterable for itself.
# Other option would be to use Mmap, if you don't want to load all data into RAM
vectors = np.load("./startup_vectors.npy")

In [82]:
payload_list[0]

{'name': 'SaferCodes',
 'images': 'https://safer.codes/img/brand/logo-icon.png',
 'alt': 'SaferCodes Logo QR codes generator system forms for COVID-19',
 'description': 'QR codes systems for COVID-19.\nSimple tools for bars, restaurants, offices, and other small proximity businesses.',
 'link': 'https://safer.codes',
 'city': 'Chicago'}

In [83]:
len(payload_list)

40474

In [84]:
len(vectors)

40474

In [88]:
client.upload_collection(
    collection_name=collection_name,
    vectors=vectors,
    payload=payload,
    ids=tqdm(range(len(payload))),
    batch_size=256,  # How many vectors will be uploaded in a single request?
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40474/40474 [00:42<00:00, 959.08it/s]
